In [1]:
import os
import sys

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

In [11]:
from tqdm import tqdm
from datasets import load_dataset
from src.decoder_utils import calculate_aggregated_hidden_states

%matplotlib inline
%config InlineBackend.figure_format='retina'

2025-09-20 15:31:46 DEBUG    matplotlib.pyplot: Loaded backend inline version unknown.


### Dataset loading

In [3]:
ds = load_dataset("ContributorsSIGNAL/SIGNAL")

2025-09-20 15:28:04 DEBUG    urllib3.connectionpool: Starting new HTTPS connection (1): huggingface.co:443
2025-09-20 15:28:04 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /datasets/ContributorsSIGNAL/SIGNAL/resolve/main/README.md HTTP/1.1" 307 0
2025-09-20 15:28:04 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /api/resolve-cache/datasets/ContributorsSIGNAL/SIGNAL/a21068568606a9b51a27e23ea3edbc78b5b6478c/README.md HTTP/1.1" 200 0
2025-09-20 15:28:04 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /datasets/ContributorsSIGNAL/SIGNAL/resolve/a21068568606a9b51a27e23ea3edbc78b5b6478c/SIGNAL.py HTTP/1.1" 404 0
2025-09-20 15:28:04 DEBUG    urllib3.connectionpool: Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-09-20 15:28:05 DEBUG    urllib3.connectionpool: https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/ContributorsSIGNAL/SIGNAL/ContributorsSIGNAL/SIGNAL.py HTTP/1.1" 404 0
2025-09-20 15:28:

In [4]:
df = ds["train"].to_pandas()

In [5]:
congruent_sentences = (df["congruent"].tolist())
congruent_sentences_unique = (df["congruent"].unique().tolist())
incongruent_sentences = (df["sentence"].tolist())

In [6]:
len(congruent_sentences), len(incongruent_sentences)

(600, 600)

In [7]:
model_names = [
    "openai-community/gpt2"
]

In [12]:
model_outputs = dict()

for model_name in tqdm(model_names, desc="Infer incongruent sentences"):
    model_outputs[model_name] = calculate_aggregated_hidden_states(
        congruent_sentences=congruent_sentences,
        incongruent_sentences=incongruent_sentences,
        model_name=model_name,
        device="mps",
    )

Infer incongruent sentences:   0%|          | 0/1 [00:00<?, ?it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

2025-09-20 15:31:52 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /openai-community/gpt2/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
2025-09-20 15:31:52 DEBUG    urllib3.connectionpool: https://huggingface.co:443 "HEAD /api/resolve-cache/models/openai-community/gpt2/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json HTTP/1.1" 200 0
/opt/miniconda3/envs/SIGNAL-analysis/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted 

[{'text': 'Автобусы проходят массовую дезинфекцией',
  'hidden_states': tensor([[[-1.3361e-01, -2.9523e-01,  1.3233e-01,  ..., -1.9327e-01,
             1.2640e-01,  4.3510e-02],
           [ 1.6545e+00, -2.3603e+00,  2.2044e+00,  ...,  8.7083e-01,
             1.8961e+00, -1.3537e+00],
           [ 1.3539e+00, -2.5095e+00,  2.0925e+00,  ...,  5.7827e-01,
             1.0882e+00, -1.8080e+00],
           ...,
           [ 8.2057e-01, -2.6083e+00,  1.1992e+00,  ...,  3.9014e-02,
             6.2009e-01, -1.9721e+00],
           [ 7.1778e-01, -2.4398e+00,  9.4018e-01,  ...,  5.2635e-02,
             9.0392e-01, -2.2020e+00],
           [-4.9651e-02, -1.3699e-01, -7.7941e-02,  ..., -3.6202e-02,
             1.0349e-01, -1.6603e-01]],
  
          [[-1.8040e-01, -2.8747e-01,  4.2283e-03,  ...,  3.7225e-02,
            -1.8909e-02, -4.3139e-02],
           [ 2.8621e-01, -1.8032e+00, -1.4992e+00,  ...,  1.1150e+00,
             9.1339e-01,  3.6814e-01],
           [ 3.1038e-01, -1.4066e+00, 